In [40]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Bidirectional, Concatenate, Flatten, Activation, RepeatVector, Permute, Multiply
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
data = pd.read_csv('Combined.csv')

# Define the numeric features and categorical features
numeric_features = ['Dur', 'TotPkts', 'TotBytes', 'SrcRate', 'DstRate']  # Modify if necessary
categorical_features = ['Proto', 'State']

# One-hot encoding for categorical features
data = pd.get_dummies(data, columns=categorical_features)

# Label encoding for the target column ('Label')
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

# Scale numeric features
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

# Prepare the input features and target labels
sequence_length = 10  # Sequence length for the model
input_features = data[numeric_features + list(data.columns.difference(numeric_features + ['Label']))].values
output_labels = data['Label'].values

# Reshape input for sequence model (samples, time_steps, features)
# Assumes data is in chronological order, otherwise sort or preprocess as needed
X = input_features.reshape(-1, sequence_length, input_features.shape[1])

# For output labels, we are predicting the last label in the sequence
y = output_labels[sequence_length - 1::sequence_length]  # Selecting every nth label based on sequence length

# Ensure that input features and output labels are float32
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Check for NaN or infinite values and handle them
if np.any(np.isnan(X)) or np.any(np.isinf(X)):
    print("NaN or infinite values detected, replacing with 0")
    X = np.nan_to_num(X)  # Replace NaNs with 0 and infinite values with large numbers


# Define the attention layer
def attention_layer(inputs):
    """
    Applies an attention mechanism on the input sequence.
    """
    attention = Dense(1, activation="tanh")(inputs)
    attention = Flatten()(attention)
    attention = Activation("softmax")(attention)
    attention = RepeatVector(inputs.shape[-1])(attention)
    attention = Permute([2, 1])(attention)
    
    output_attention = Multiply()([inputs, attention])
    return output_attention

# Define model input
input_layer = Input(shape=(sequence_length, X.shape[2]))  # (time_steps, features)

# LSTM branch
lstm_branch = LSTM(64, activation='relu', return_sequences=True)(input_layer)
lstm_branch = LSTM(32, activation='relu', return_sequences=True)(lstm_branch)

# Bidirectional LSTM branch
bidirectional_branch = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_layer)
bidirectional_branch = Bidirectional(LSTM(32, activation='relu', return_sequences=True))(bidirectional_branch)

# GRU branch
gru_branch = GRU(64, activation='relu', return_sequences=True)(input_layer)
gru_branch = GRU(32, activation='relu', return_sequences=True)(gru_branch)

# Concatenate all branches
combined = Concatenate()([lstm_branch, bidirectional_branch, gru_branch])

# Apply attention mechanism
attention_output = attention_layer(combined)

# Flatten and output layers
flattened = Flatten()(attention_output)
dense_output = Dense(64, activation='relu')(flattened)
dense_output = Dense(32, activation='relu')(dense_output)
final_output = Dense(1, activation='sigmoid')(dense_output)  # Sigmoid for binary classification

# Define the model
model = Model(inputs=input_layer, outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
history = model.fit(
    X, y,
    epochs=10,
    batch_size=32,
    validation_split=0.2,  # Automatically split 20% of the data for validation
    verbose=1
)


C:\Users\ANANT TIWARI\AppData\Local\Temp\ipykernel_16068\3226047993.py:8: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Combined.csv')


ValueError: could not convert string to float: 'Benign'

In [46]:
import pandas as pd
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Bidirectional, Concatenate, Flatten, Activation, RepeatVector, Permute, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [47]:
data= pd.read_csv('Combined.csv')

C:\Users\ANANT TIWARI\AppData\Local\Temp\ipykernel_16068\3381191799.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('Combined.csv')


In [59]:
data.columns

Index(['Unnamed: 0', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max',
       'sTos', 'dTos', 'sDSb', 'dDSb', 'sTtl', 'dTtl', 'sHops', 'dHops',
       'Cause', 'TotPkts', 'SrcPkts', 'DstPkts', 'TotBytes', 'SrcBytes',
       'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'Load', 'SrcLoad',
       'DstLoad', 'Loss', 'SrcLoss', 'DstLoss', 'pLoss', 'SrcGap', 'DstGap',
       'Rate', 'SrcRate', 'DstRate', 'SrcWin', 'DstWin', 'sVid', 'dVid',
       'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat', 'Label',
       'Attack Type', 'Attack Tool', 'Proto_arp', 'Proto_icmp',
       'Proto_ipv6-icmp', 'Proto_llc', 'Proto_lldp', 'Proto_sctp', 'Proto_tcp',
       'Proto_udp', 'State_ACC', 'State_CON', 'State_ECO', 'State_FIN',
       'State_INT', 'State_NRS', 'State_REQ', 'State_RSP', 'State_RST',
       'State_TST', 'State_URP'],
      dtype='object')

In [65]:
categorical_features = ['sDSb', 'dDSb', 'Cause', 'Attack Type', 'Attack Tool', 
                        'Proto_arp', 'Proto_icmp', 'Proto_ipv6-icmp', 'Proto_llc', 
                        'Proto_lldp', 'Proto_sctp', 'Proto_tcp', 'Proto_udp', 
                        'State_ACC', 'State_CON', 'State_ECO', 'State_FIN', 'State_INT', 
                        'State_NRS', 'State_REQ', 'State_RSP', 'State_RST', 'State_TST', 'State_URP']

# Check if all categorical features exist in the dataset
missing_columns = [col for col in categorical_features if col not in data.columns]

In [67]:
categorical_features = ['sDSb', 'dDSb', 'Cause', 'Attack Type', 'Attack Tool', 
                        'Proto_arp', 'Proto_icmp', 'Proto_ipv6-icmp', 'Proto_llc', 
                        'Proto_lldp', 'Proto_sctp', 'Proto_tcp', 'Proto_udp', 
                        'State_ACC', 'State_CON', 'State_ECO', 'State_FIN', 'State_INT', 
                        'State_NRS', 'State_REQ', 'State_RSP', 'State_RST', 'State_TST', 'State_URP']

# Check which categorical columns are actually in the dataset
missing_columns = [col for col in categorical_features if col not in data.columns]
print(f"Missing columns: {missing_columns}")

# Remove missing columns from categorical_features
categorical_features = [col for col in categorical_features if col in data.columns]

# Apply one-hot encoding only for the available categorical columns
data = pd.get_dummies(data, columns=categorical_features)

# Check the updated dataset columns
print(data.columns)

Missing columns: ['sDSb', 'dDSb', 'Cause', 'Attack Type', 'Attack Tool']
Index(['Unnamed: 0', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max',
       'sTos', 'dTos',
       ...
       'State_REQ_False', 'State_REQ_True', 'State_RSP_False',
       'State_RSP_True', 'State_RST_False', 'State_RST_True',
       'State_TST_False', 'State_TST_True', 'State_URP_False',
       'State_URP_True'],
      dtype='object', length=120)


In [69]:
numeric_features = ['Dur', 'TotPkts', 'TotBytes', 'SrcRate', 'DstRate', 'RunTime', 'Mean', 'Sum', 'Min', 'Max', 
                    'sTos', 'dTos', 'sTtl', 'dTtl', 'sHops', 'dHops', 'TotPkts', 'SrcPkts', 'DstPkts', 
                    'SrcBytes', 'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'Load', 'SrcLoad', 'DstLoad',
                    'Loss', 'SrcLoss', 'DstLoss', 'pLoss', 'SrcGap', 'DstGap', 'Rate', 'SrcWin', 'DstWin', 
                    'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat']  # Ensure numeric columns only


In [66]:
# numeric_features = ['Dur', 'TotPkts', 'TotBytes', 'SrcRate', 'DstRate']  # Add other relevant numeric columns
# categorical_features = ['sDSb','dDSb','Cause','Label','Attack Type','Attack Tool']

# # Encode categorical features
# data = pd.get_dummies(data, columns=categorical_features)

numeric_features = ['Dur', 'TotPkts', 'TotBytes', 'SrcRate', 'DstRate', 'RunTime', 'Mean', 'Sum', 'Min', 'Max', 
                    'sTos', 'dTos', 'sTtl', 'dTtl', 'sHops', 'dHops', 'TotPkts', 'SrcPkts', 'DstPkts', 
                    'SrcBytes', 'DstBytes', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'Load', 'SrcLoad', 'DstLoad',
                    'Loss', 'SrcLoss', 'DstLoss', 'pLoss', 'SrcGap', 'DstGap', 'Rate', 'SrcWin', 'DstWin', 
                    'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat']  # Ensure numeric columns only

# Define categorical columns (excluding 'Label')
categorical_features = ['sDSb', 'dDSb', 'Cause', 'Attack Type', 'Attack Tool', 
                        'Proto_arp', 'Proto_icmp', 'Proto_ipv6-icmp', 'Proto_llc', 
                        'Proto_lldp', 'Proto_sctp', 'Proto_tcp', 'Proto_udp', 
                        'State_ACC', 'State_CON', 'State_ECO', 'State_FIN', 'State_INT', 
                        'State_NRS', 'State_REQ', 'State_RSP', 'State_RST', 'State_TST', 'State_URP']

# One-hot encode categorical features (excluding 'Label')
data = pd.get_dummies(data, columns=categorical_features)

KeyError: "['sDSb', 'dDSb', 'Cause', 'Attack Type', 'Attack Tool'] not in index"

In [73]:
categorical_features = ['sDSb', 'dDSb', 'Cause', 'Attack Type', 'Attack Tool', 
                        'Proto_arp', 'Proto_icmp', 'Proto_ipv6-icmp', 'Proto_llc', 
                        'Proto_lldp', 'Proto_sctp', 'Proto_tcp', 'Proto_udp', 
                        'State_ACC', 'State_CON', 'State_ECO', 'State_FIN', 'State_INT', 
                        'State_NRS', 'State_REQ', 'State_RSP', 'State_RST', 'State_TST', 'State_URP']

# Remove missing columns from categorical_features
categorical_features = [col for col in categorical_features if col in data.columns]

# One-hot encode categorical features (excluding 'Label')
data = pd.get_dummies(data, columns=categorical_features)

# Verify the updated columns
print("Updated Columns after one-hot encoding:", data.columns)

Updated Columns after one-hot encoding: Index(['Unnamed: 0', 'Seq', 'Dur', 'RunTime', 'Mean', 'Sum', 'Min', 'Max',
       'sTos', 'dTos',
       ...
       'State_REQ_False', 'State_REQ_True', 'State_RSP_False',
       'State_RSP_True', 'State_RST_False', 'State_RST_True',
       'State_TST_False', 'State_TST_True', 'State_URP_False',
       'State_URP_True'],
      dtype='object', length=120)


In [76]:
for i in range(len(data.columns)):
    print(data.columns[i])
    print("\n")

Unnamed: 0


Seq


Dur


RunTime


Mean


Sum


Min


Max


sTos


dTos


sTtl


dTtl


sHops


dHops


TotPkts


SrcPkts


DstPkts


TotBytes


SrcBytes


DstBytes


Offset


sMeanPktSz


dMeanPktSz


Load


SrcLoad


DstLoad


Loss


SrcLoss


DstLoss


pLoss


SrcGap


DstGap


Rate


SrcRate


DstRate


SrcWin


DstWin


sVid


dVid


SrcTCPBase


DstTCPBase


TcpRtt


SynAck


AckDat


sDSb_39


sDSb_4


sDSb_52


sDSb_54


sDSb_af11


sDSb_af12


sDSb_af41


sDSb_cs0


sDSb_cs4


sDSb_cs6


sDSb_cs7


sDSb_ef


dDSb_af11


dDSb_af12


dDSb_cs0


dDSb_cs1


dDSb_cs4


dDSb_ef


Cause_Shutdown


Cause_Start


Cause_Status


Label_0


Label_1


Attack Type_Benign


Attack Type_HTTPFlood


Attack Type_ICMPFlood


Attack Type_SYNFlood


Attack Type_SYNScan


Attack Type_SlowrateDoS


Attack Type_TCPConnectScan


Attack Type_UDPFlood


Attack Type_UDPScan


Attack Tool_Benign


Attack Tool_Goldeneye


Attack Tool_Hping3


Attack Tool_Nmap


Attack Tool_Slowloris


Attack Tool_Torshamme

In [78]:
label_encoder = LabelEncoder()

# Assuming 'Label_0' and 'Label_1' are categorical labels
data['Label_0'] = label_encoder.fit_transform(data['Label_0'])
data['Label_1'] = label_encoder.fit_transform(data['Label_1'])

# Scale numeric features
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

In [82]:
label_encoder = LabelEncoder()

# Use one of the labels for binary classification
data['Label'] = label_encoder.fit_transform(data['Label_1'])  # Or 'Label_1'

In [83]:
# Define input and output for the model
sequence_length = 10  # Define sequence length
input_features = data[numeric_features + list(data.columns.difference(numeric_features + ['Label_1']))].values
output_labels = data['Label_1'].values

In [84]:
data

,Unnamed: 0,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,dTos,...,State_REQ_True,State_RSP_False,State_RSP_True,State_RST_False,State_RST_True,State_TST_False,State_TST_True,State_URP_False,State_URP_True,Label
0,0,1,-0.806980,-0.806980,-0.806980,-0.806980,-0.806980,-0.806980,-0.069046,NaN,...,False,True,False,True,False,True,False,True,False,0
1,1,2,-0.806980,-0.806980,-0.806980,-0.806980,-0.806980,-0.806980,-0.069046,NaN,...,False,True,False,True,False,True,False,True,False,0
2,2,3,2.148165,2.148165,2.148165,2.148165,2.148165,2.148165,-0.069046,-0.125376,...,False,True,False,True,False,True,False,True,False,0
3,3,4,2.148175,2.148175,2.148175,2.148175,2.148175,2.148175,-0.069046,-0.125376,...,False,True,False,True,False,True,False,True,False,0
4,4,5,2.149013,2.149013,2.149013,2.149013,2.149013,2.149013,-0.069046,-0.125376,...,False,True,False,True,False,True,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215885,487569,1,-0.806980,-0.806980,-0.806980,-0.806980,-0.806980,-0.806980,14.788432,8.716014,...,False,True,False,True,False,True,False,True,False,0
1215886,487570,3,-0.667674,-0.667674,-0.667674,-0.667674,-0.667674,-0.667674,14.788432,1.775998,...,False,True,False,True,False,True,False,True,False,0
1215887,487571,764,-0.747897,-0.747897,-0.747897,-0.747897,-0.747897,-0.747897,-0.069046,-0.125376,...,False,True,False,True,False,True,False,True,False,0
1215888,487572,3,-0.033695,-0.033695,-0.033695,-0.033695,-0.033695,-0.033695,14.788432,1.775998,...,False,True,False,True,False,True,False,True,False,0


In [85]:
# Reshape input for sequence model (samples, time_steps, features)
# Assumes data is in chronological order, otherwise sort or preprocess as needed
X = input_features.reshape(-1, sequence_length, input_features.shape[1])
y = output_labels.reshape(-1, sequence_length)[:, -1]  # Predict based on last label in the sequence


In [86]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)
# Check for NaN or infinite values and handle them
if np.any(np.isnan(X)) or np.any(np.isinf(X)):
    print("NaN or infinite values detected, replacing with 0")
    X = np.nan_to_num(X)  # Replace NaNs with 0 and infinite values with large numbers


NaN or infinite values detected, replacing with 0


In [87]:
def attention_layer(inputs):
    """
    Applies an attention mechanism on the input sequence.
    """
    attention = Dense(1, activation="tanh")(inputs)
    attention = Flatten()(attention)
    attention = Activation("softmax")(attention)
    attention = RepeatVector(inputs.shape[-1])(attention)
    attention = Permute([2, 1])(attention)
    
    output_attention = Multiply()([inputs, attention])
    return output_attention

In [88]:
# Define model input
input_layer = Input(shape=(sequence_length, X.shape[2]))  # (time_steps, features)


In [89]:
# LSTM branch
lstm_branch = LSTM(64, activation='relu', return_sequences=True)(input_layer)
lstm_branch = LSTM(32, activation='relu', return_sequences=True)(lstm_branch)

# Bidirectional LSTM branch
bidirectional_branch = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_layer)
bidirectional_branch = Bidirectional(LSTM(32, activation='relu', return_sequences=True))(bidirectional_branch)

# GRU branch
gru_branch = GRU(64, activation='relu', return_sequences=True)(input_layer)
gru_branch = GRU(32, activation='relu', return_sequences=True)(gru_branch)

In [90]:
# Concatenate all branches
combined = Concatenate()([lstm_branch, bidirectional_branch, gru_branch])

# Apply attention mechanism
attention_output = attention_layer(combined)

In [91]:
# Flatten and output layers
flattened = Flatten()(attention_output)
dense_output = Dense(64, activation='relu')(flattened)
dense_output = Dense(32, activation='relu')(dense_output)
final_output = Dense(1, activation='sigmoid')(dense_output)  # Sigmoid for binary classification

# Define the model
model = Model(inputs=input_layer, outputs=final_output)


In [92]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [93]:
# Display model summary
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 10, 121)]            0         []                            
                                                                                                  
 lstm_20 (LSTM)              (None, 10, 64)               47616     ['input_5[0][0]']             
                                                                                                  
 bidirectional_10 (Bidirect  (None, 10, 128)              95232     ['input_5[0][0]']             
 ional)                                                                                           
                                                                                                  
 gru_10 (GRU)                (None, 10, 64)               35904     ['input_5[0][0]']       

In [96]:
history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    validation_split=0.2,  # Automatically split 20% of the data for validation
    verbose=1
)

Epoch 1/50
3040/3040 [==============================] - 91s 30ms/step - loss: 0.8764 - accuracy: 0.7701 - val_loss: 0.6342 - val_accuracy: 0.6686
Epoch 2/50
3040/3040 [==============================] - 81s 27ms/step - loss: 0.5194 - accuracy: 0.8062 - val_loss: 0.5769 - val_accuracy: 0.7990
Epoch 3/50
3040/3040 [==============================] - 83s 27ms/step - loss: 0.4540 - accuracy: 0.8308 - val_loss: 0.6725 - val_accuracy: 0.6327
Epoch 4/50
3040/3040 [==============================] - 149s 49ms/step - loss: 0.4218 - accuracy: 0.8430 - val_loss: 0.6122 - val_accuracy: 0.7900
Epoch 5/50
3040/3040 [==============================] - 79s 26ms/step - loss: 0.3265 - accuracy: 0.8580 - val_loss: 0.6607 - val_accuracy: 0.7809
Epoch 6/50
3040/3040 [==============================] - 80s 26ms/step - loss: 0.5874 - accuracy: 0.7503 - val_loss: 0.6426 - val_accuracy: 0.7990
Epoch 7/50
3040/3040 [==============================] - 82s 27ms/step - loss: 0.6850 - accuracy: 0.5593 - val_loss: 0.6236 

In [97]:
model.save('model6.h5')

C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
